# QUBO and Variational Quantum Eigensolvers (VQE)

$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right|}$
$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$
$\newcommand{\expval}[1]{\left\langle{#1}\right\rangle}$
$\DeclareMathOperator*{\argmax}{arg\,max}$
$\DeclareMathOperator*{\argmin}{arg\,min}$
## QUBO problem definition

QUBO problems, such as the infamous Max-Cut, could generally be defined as follows: 

$$\min_{x \in \{0,1\}^n} x^T Q x + \mu^T x$$

where $x$ is an $n$-dimensional binary vector, the QUBO matrix $Q$ is an $n \times n$ square matrix, and the QUBO vector $\mu$ is an $n$-dimensional vector. Having $Q$ and $\mu$ for the problem at hand, the objective is to find the binary vector $x$ which minimizes the above expression.

In the case of the Max-Cut problem, $x$ represents the binary coloring scheme of the graph nodes, i.e. $x_i$ would be $0$/$1$ if node $i$ is assigned to set $0$/set $1$. A relevant cost function to be minimized could then be the sum of weights of the edges that connect two nodes from the same set: 

$$\sum_{i,j=1}^n x_i w_{ij} x_j + \sum_{i,j=1}^n (1-x_i)w_{ij}(1-x_j).$$ 

However, we could also formulate Max-Cut as a maximization problem and then transform it into a minimization problem. It suffices to sum the weights of edges connecting node $i$ from set $1$ to node $j$ from set $0$. This summation should be maximized; however, if we negate this summation, we should minimize. Consequently, the corresponding cost function to be minimized as in a QUBO generic from would be:

$$C(x) = - \sum_{i,j=1}^n x_i w_{ij} (1 - x_j) = - x^T w x + (wx)^T x$$

in which $-w$ and $wx$ would be the QUBO matrix and the QUBO vector respectively.

## Formulating QUBO as a ground state problem

In quantum mechanics, Hamiltonian ($\hat{H}$) is responsible for the time-evolution of a system. In other words, if you know the Hamiltonian of a system and its initial state, the state of the system could be determined in any other time. One should note that the Hamiltonian of a system could be _time-dependent_. Furthermore, $H$'s might not _commute_ at different times : $[\hat{H}(t_1),\hat{H}(t_2)] \ne 0$.

Here, we are interested in finding the Hamiltonian $\hat{H}_C$ whose eigenstates are $\ket{x}$ with eigenvalues $C(x)$.

$$\hat{H}_C \ket{x} = C(x) \ket{x}$$

The _ground state_ of this Hamiltonian would then be the bound state with the minimum $C(x)$. In other words, any QUBO problem is equivalent to finding the ground state of a Hamiltonian $\hat{H}_C$. But how to find $\hat{H}_C$ for instance for Max-Cut?

First, we have to select a basis for $\hat{H}_C$ representation. Any set of orthogonal basis vectors in the corresponding Hilbert space could be selected. $Z$-bases with eigenvalues $1$ and $-1$ are convenient for now:

$$\hat{Z}_i \ket{x} = z_i \ket{x}, \quad z_i=(-1)^{x_i}~~for~~x_i \in \{0,1\}$$

If we represent $C(x)$ in terms of $z_i$'s, the analogous representation of $\hat{H}_C$ in the $Z$-bases would be determined. A possible change of variable for this is 

$$ x_i = \frac{1-z_i}{2}.$$

Consequently, the corresponding Hamiltonian in terms of the Pauli $\hat{Z}$ operators would take the form:

$$ \hat{H}_C = - \frac{1}{2} \sum_{i<j} w_{ij} \hat{Z}_i \hat{Z}_j + const.$$

and our QUBO problem is equivalent to finding a state $\ket{\psi} = \sum_x c_x \ket{x}$ that minimizes the expectation value of our _Ising Hamiltonian_.: 

$$\expval{\hat{H}_C} = \frac{\bra{\psi}\hat{H}_C\ket{\psi}}{\braket{\psi}{\psi}}.$$

In order to be able to compare the average energy levels of arbitrary states, we should use the expectation value instead of eigenvalue; firstly, because the arbitrary state might not be an eigenstate of our Hamiltonian and secondly, it might not be normalized. 

It is straightforward to check that the state that minimizes the above expectation value is the ground state of our system. However, finding the minimal energy of complicated Hamiltonians are analytically impossible and one should tap into _perturbative calculations_ and other approximation methodologies to attack this problem.


## Variational Quantum Eigensolver (VQE)

Consider the Hamiltonian $\hat{H}_C$ and $E_0$ as the ground state energy of this Hamiltonian. It is worth recapping the seemingly trivial but extremely powerful _variational principle of quantum mechanics_. 


> __Variational Principle of Quantum Mechanics__
>
>For any arbitrary _trial state_ $\ket{\psi}$, the average energy level of the state is greater than or equal to $E_0$:
>
>$$\expval{\hat{H}_C}_{\psi} \ge E_0$$


The objective is to start from an initial trial state, evaluate its energy, and step-by-step modify the state in a way that takes us closer and closer to the ground state. But how should we modify the trial states to get closer to the optimal solution (the ground state)? We should do unitary state modifications in the corresponding Hilbert space (another name for doing rotations and inversions in the Hilbert space); however, a guideline is needed to tell us the direction and amount of change at each step. 

### Parameterization (ansatz circuit or variational from)

The required guideline could be formulated on the basis of a set of optimization parameters $\theta$:
- Trial states need to be parameterized. This means that the trial state now becomes $\ket{\psi(\theta)}$ and it would be modified on the basis of the changes we make in $\theta$.

$$\ket{\psi(\theta)} = \hat{U}(\theta)\ket{0}$$

- _Parameterized_ quantum circuits (quantum operators) would be required as agents of change. These parameterized circuits would prepare $\ket{\psi(\theta)}$ at the beginning of each iteration and evaluate $\expval{\hat{H}_C}_{\psi(\theta)}$.

> __Generic Near-Term Parameterization__
>
> Parameterization for near-term quantum computers is mainly achieved starting by fixed/parameterized single-qubit rotations on all qubits at the entrance of the circuit followed by fixed/parameterized two-qubit entangling gates. The final structure could have layers, blocks, or trees.


### Algorithm

Variational Quantum Eigensolvers are a set of hybrid quantum-classical algorithms specifically developed for near-term quantum computers. The cornerstone is parametrization :

0) parameterize trial states and the corresponding quantum circuit; objective is to find the optimal set of parameters:

$$ \theta_{opt} = {\argmin}_{\theta} \expval{\hat{H}_C}_{\psi(\theta)}$$

> __Parameterizarion Quality__ 
>
> The exact method of parametrization mainly depends on quantum hardware considerations and optimization performance factors. In the end, the quality of the result would depend on this parametrization.

At iteration $n$ of the algorithm do the following:

1) __evaluate energy__ $\to$ setup the quantum circuit with $\theta_n$ and evaluate the energy of the trial state $\ket{\psi(\theta_n)}$ using the circuit.


2) __optimize $\theta$__ $\to$ on a classical computer, run a classical optimization algorithm such as _gradient descent_, _natural gradient_, _sampling gradient_ and so forth. Generally, these algorithms would take into account the previous iterations results to update $\theta$, i.e. to find $\theta_{n+1}$ for the next iteration quantum circuit setup.


## Example

As an example, we will implement a VQE for solving a MaxCut using QisKit predefined [`EfficientSU2`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.EfficientSU2.html#qiskit.circuit.library.EfficientSU2) as the ansatz ([N-local circuit](https://qiskit.org/documentation/apidoc/circuit_library.html)).

In [1]:
# install qiskit_optimizarion if ModuleNotFound

import sys
import subprocess

try:
    import qiskit_optimization
except ModuleNotFoundError:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', 'qiskit_optimization'], stdout=subprocess.DEVNULL)

### Generate random undirected graph

In [2]:
import networkx as nx
import numpy as np
import random
#
# generate random undirected graph with 
# n-verices and m-edges
#
n = 5 
m = 7 
g = nx.gnm_random_graph(n, m)
for (u, v, w) in g.edges(data=True):
    w['weight'] = random.randint(1, 5)
#
# print edges data
#
c = 0
for edge in g.edges.data():
    c += 1
    print("edge %d : connecting %d to %d - %s" % (c, edge[0], edge[1], edge[2]))
#
# construct the weight matrix
#
w = np.zeros([n, n])
for i in range(n):
    for j in range(n):
        temp = g.get_edge_data(i, j, default=0)
        if temp != 0:
            w[i, j] = temp["weight"]
print('-'*60)
print('weight matrix : ')
print(w)
print('-'*60)

edge 1 : connecting 0 to 2 - {'weight': 3}
edge 2 : connecting 0 to 4 - {'weight': 5}
edge 3 : connecting 0 to 3 - {'weight': 4}
edge 4 : connecting 0 to 1 - {'weight': 5}
edge 5 : connecting 1 to 2 - {'weight': 1}
edge 6 : connecting 2 to 4 - {'weight': 5}
edge 7 : connecting 2 to 3 - {'weight': 3}
------------------------------------------------------------
weight matrix : 
[[0. 5. 3. 4. 5.]
 [5. 0. 1. 0. 0.]
 [3. 1. 0. 3. 5.]
 [4. 0. 3. 0. 0.]
 [5. 0. 5. 0. 0.]]
------------------------------------------------------------


### Find the Ising Hamiltonian

In [3]:
from qiskit_optimization.applications import Maxcut

maxcut = Maxcut(g)
qp = maxcut.to_quadratic_program()
print(qp.prettyprint())

hamiltonian, offset = qp.to_ising()
#
# the coeffs of Z_i*Z_j in the Ising Hamiltonian would be 
# a quarter of the corresponding coeffs in qp.
#
print('The corresponding Ising Hamiltonian: \n',hamiltonian,'\n+ (',offset,')')

Problem name: Max-cut

Maximize
  -10*x_0*x_1 - 6*x_0*x_2 - 8*x_0*x_3 - 10*x_0*x_4 - 2*x_1*x_2 - 6*x_2*x_3
  - 10*x_2*x_4 + 17*x_0 + 6*x_1 + 12*x_2 + 7*x_3 + 10*x_4

Subject to
  No constraints

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4

The corresponding Ising Hamiltonian: 
 2.5 * IIIZZ
+ 1.5 * IIZIZ
+ 0.5 * IIZZI
+ 2.0 * IZIIZ
+ 1.5 * IZZII
+ 2.5 * ZIIIZ
+ 2.5 * ZIZII 
+ ( -13.0 )


### Prepare VQE

We are going to use EfficientSU2 for parameterization; however, any customized ansatz could have been defined and used instead. 

In [4]:
#
# use EfficientSU2 as the ansatz circuit
#
from qiskit.circuit.library.n_local import EfficientSU2
#
# deploy Estimator to evaluate expectation values
#
from qiskit.primitives import Estimator
#
# deploy GSLS as the classical local optimizer
#
from qiskit.algorithms.optimizers import GSLS
#
# deploy VQE as the variational algorithm
#
from qiskit.algorithms.minimum_eigensolvers import VQE

ansatz = EfficientSU2(num_qubits=n, reps=2)

vqe = VQE(estimator=Estimator(), ansatz=ansatz, optimizer=GSLS())

print(ansatz.decompose())

     ┌──────────┐┌──────────┐                                            »
q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[5]) ├─────────────────────────────────────■──────»
     ├──────────┤├──────────┤                                   ┌─┴─┐    »
q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[6]) ├────────────────────────■──────────┤ X ├────»
     ├──────────┤├──────────┤                      ┌─┴─┐    ┌───┴───┴───┐»
q_2: ┤ Ry(θ[2]) ├┤ Rz(θ[7]) ├───────────■──────────┤ X ├────┤ Ry(θ[12]) ├»
     ├──────────┤├──────────┤         ┌─┴─┐    ┌───┴───┴───┐├───────────┤»
q_3: ┤ Ry(θ[3]) ├┤ Rz(θ[8]) ├──■──────┤ X ├────┤ Ry(θ[13]) ├┤ Rz(θ[18]) ├»
     ├──────────┤├──────────┤┌─┴─┐┌───┴───┴───┐├───────────┤└───────────┘»
q_4: ┤ Ry(θ[4]) ├┤ Rz(θ[9]) ├┤ X ├┤ Ry(θ[14]) ├┤ Rz(θ[19]) ├─────────────»
     └──────────┘└──────────┘└───┘└───────────┘└───────────┘             »
«     ┌───────────┐┌───────────┐                          ┌───────────┐»
«q_0: ┤ Ry(θ[10]) ├┤ Rz(θ[15]) ├───────────────────■──────┤ Ry(θ[20]) ├»
«     ├───────────┤├─────────

### Run VQE for the problem's Hamiltonian

This might take some time depending on the number of parameters used by the ansatz.

In [5]:
result = vqe.compute_minimum_eigenvalue(hamiltonian)

In [6]:
print('-'*60)
print('iterations : ', result.cost_function_evals)
print('optimizer time : ', result.optimizer_time)
print('found optimal energy : ', result.optimal_value.real + offset)
print('-'*60)

------------------------------------------------------------
iterations :  3132
optimizer time :  26.498186349868774
found optimal energy :  -22.98441386110113
------------------------------------------------------------


The object `result` embodies circuit `optimal_parameters` and the _energy_ of the optimal trial state (`optimal_value`). In order to find the optimal trial state, we should do _sampling_. This could have been done by deploying `SamplingVQE` instead of `VQE` which would do an automatic sampling when optimal parameters are obtained. Now that we have the optimal circuit parameters, we could perform the sampling on the circuit ourselves.

In [7]:
from qiskit.primitives import Sampler

optimal_state = ansatz.bind_parameters(result.optimal_parameters)
optimal_state.measure_all()

sampler = Sampler()
distribution = sampler.run([optimal_state]).result().quasi_dists[0]

solution = max(distribution.binary_probabilities().items(), key=lambda x: x[1])

for entry in distribution.binary_probabilities().items():
    print("state : %s , probability : %f" % (entry[0], entry[1]))

print('-'*60)
print('Optimal State : ', solution[0])
print('Probability : ', solution[1])
print('-'*60)

state : 00000 , probability : 0.000000
state : 00001 , probability : 0.000000
state : 00010 , probability : 0.000000
state : 00011 , probability : 0.000000
state : 00100 , probability : 0.000000
state : 00101 , probability : 0.000000
state : 00110 , probability : 0.000000
state : 00111 , probability : 0.000006
state : 01000 , probability : 0.000000
state : 01001 , probability : 0.000000
state : 01010 , probability : 0.000006
state : 01011 , probability : 0.000000
state : 01100 , probability : 0.000000
state : 01101 , probability : 0.000000
state : 01110 , probability : 0.000000
state : 01111 , probability : 0.000000
state : 10000 , probability : 0.000000
state : 10001 , probability : 0.000000
state : 10010 , probability : 0.000004
state : 10011 , probability : 0.000000
state : 10100 , probability : 0.000000
state : 10101 , probability : 0.000000
state : 10110 , probability : 0.000000
state : 10111 , probability : 0.000000
state : 11000 , probability : 0.000927
state : 11001 , probabili